In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import AnubisSuperScript as ass
from functools import partial
import importlib
from tqdm import tqdm
importlib.reload(ass)
import ast

ERROR:root:Cannot activate multiple GUI eventloops


In [2]:

mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

strips_on_steroid = {
    'rpc0': {'phi': [0]},
    'rpc1': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31,0], 'phi': [0]},
    'rpc5': {'eta': [31,0], 'phi': [0]},
}


In [ ]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\proAnubis_240403_0227.h5')

In [3]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\threeHourRun_24_3_1.h5')
thisData = [sublist[:94230] for sublist in thisData]

In [493]:
thisData = [sublist[94230:188460] for sublist in thisData]

In [4]:
etaHits, phiHits = ass.divideHitCountsByRPC_Timed(thisData)

In [5]:
df = ass.remake_data(thisData, mapping)

In [6]:
df_clean = df[df['time'] <= 500]

In [7]:
cluster_df = ass.calculate_cluster_metrics_better(df_clean, 5)

In [11]:
show(cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [8]:
def filter_groups(group):
    has_eta = 'eta' in group['strip_direction'].values
    has_phi = 'phi' in group['strip_direction'].values
    return has_eta and has_phi

filtered_cluster = cluster_df.groupby(['event_number', 'rpc_number']).filter(filter_groups)

In [13]:
print(filtered_cluster.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697379 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     697379 non-null  int64 
 1   rpc_number       697379 non-null  object
 2   strip_direction  697379 non-null  object
 3   locations        697379 non-null  object
 4   times            697379 non-null  object
 5   start_time       697379 non-null  int64 
 6   end_time         697379 non-null  int64 
 7   size             697379 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 47.9+ MB
None


In [18]:
show(filtered_cluster.sort_values(by=['event_number', 'rpc_number']))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [10]:
print(cluster_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839727 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     839727 non-null  int64 
 1   rpc_number       839727 non-null  object
 2   strip_direction  839727 non-null  object
 3   locations        839727 non-null  object
 4   times            839727 non-null  object
 5   start_time       839727 non-null  int64 
 6   end_time         839727 non-null  int64 
 7   size             839727 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 51.3+ MB
None


In [9]:
fully_massaged_df = filtered_cluster[filtered_cluster['size'] <= 5]

In [16]:
print(fully_massaged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 675851 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     675851 non-null  int64 
 1   rpc_number       675851 non-null  object
 2   strip_direction  675851 non-null  object
 3   locations        675851 non-null  object
 4   times            675851 non-null  object
 5   start_time       675851 non-null  int64 
 6   end_time         675851 non-null  int64 
 7   size             675851 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 46.4+ MB
None


In [518]:
show(fully_massaged_df.sort_values(by = ['event_number', 'strip_direction']))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [15]:
print(len(fully_massaged_df))

675851


In [431]:

def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])  
    weights = 1 / (uncertainties)**2

    try:
        coeffs= np.polyfit(heights, locations, 1, cov=False, w=weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]
        # cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])

    slope, intercept = coeffs
    # slope_error, intercept_error = np.sqrt(np.diag(cov))
    slope_error, intercept_error = 0, 0
    predicted = slope * heights + intercept
    residuals = locations - predicted
    RSS = np.sum(residuals ** 2)

    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df,n):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.8, 
        'rpc2': 1.6, 
        'rpc3': 61.6, 
        'rpc4': 120.8, 
        'rpc5': 121.6
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        if len(event_group) > 1:
            continue
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < n:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            for combo_size in range(n, len(unique_rpcs) + 1):
                for combo in combinations(all_clusters, combo_size):
                    if len({c['rpc'] for c in combo}) != len(combo):
                        continue  # Skip combinations where RPCs are not unique

                    time_diffs_are_valid = True
                    for i in range(len(combo)):
                        for j in range(i + 1, len(combo)):
                            error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                            height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                            time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                            expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                            uncertainty_margin = 3

                            if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                                time_diffs_are_valid = False
                                break
                        if not time_diffs_are_valid:
                            break

                    if not time_diffs_are_valid:
                        continue  # Skip to the next combination if time differences are invalid

                    # Calculate metrics only for valid combinations
                    metric = calculate_metric_for_combo(combo, rpc_heights)
                    if metric[4] <= 2:
                        valid_combinations.append(metric)
                        

                for valid_combination in valid_combinations:
                    paths.append({
                        'Event Number': event_number,
                        'Direction': direction,
                        'Slope': valid_combination[0],
                        'Intercept': valid_combination[1],
                        'Slope_error': valid_combination[2],
                        'Intercept_error': valid_combination[3],
                        'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                        'RSS': valid_combination[4],
                    })

    path_df = pd.DataFrame(paths)
    return path_df

In [60]:
height = np.array([1,2,3,4])
print(5 * height)

[ 5 10 15 20]


In [432]:
scale_eta = 2.5
scale_phi = 2.5
var_eta = scale_eta ** 2 / 12  # Define the variance for eta
var_phi = scale_phi ** 2 / 12  # Define the variance for phi

def calculate_metric_for_combo(combo, rpc_heights, direction):
    # Extracting and scaling coordinates based on the direction
    # scale_factor = scale_eta if direction == 'eta' else scale_phi

    # Parse coordinates and scale
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])
    weights = 1 /  (2 *uncertainties)
    try:
        coeffs, res, _, _, _ = np.polyfit(heights, locations, 1, full=True, w = weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]  # Handle cases where the fit cannot be computed

    slope, intercept = coeffs
    predicted = slope * heights + intercept
    RSS = res ** 2

    sigma_squared = var_eta if direction == 'eta' else var_phi
    chi_squared = np.sum(((res) ** 2) / sigma_squared)
    chi_squared_norm = chi_squared / (len(combo)-2) 

    return slope, intercept, chi_squared_norm, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df,n):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (0, 2),
    ('rpc0', 'phi'): (0, 2),
    ('rpc1', 'eta'): (0, 2),
    ('rpc1', 'phi'): (0, 2),
    ('rpc2', 'eta'): (0, 2),
    ('rpc2', 'phi'): (0, 2),
    ('rpc3', 'eta'): (0, 2),
    ('rpc3', 'phi'): (0, 2),
    ('rpc4', 'eta'): (0, 2),
    ('rpc4', 'phi'): (0, 2),
    ('rpc5', 'eta'): (0, 2),
    ('rpc5', 'phi'): (0, 2),
}
    paths = []

    adjusted_muon_speed_cm_ns = 15

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.8, 
        'rpc2': 1.6, 
        'rpc3': 61.6, 
        'rpc4': 120.8, 
        'rpc5': 121.6
    }


    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        if len(event_group) > 50:
            continue
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < n:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = scale_eta if direction == 'eta' else scale_phi
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    event_time_diff = (np.max(row['times']) - np.min(row['times'])) / len(row['times'])
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'event_time_diff': event_time_diff,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            for combo_size in range(n, len(unique_rpcs) + 1):
                for combo in combinations(all_clusters, combo_size):
                    if len({c['rpc'] for c in combo}) != len(combo):
                        continue  # Skip combinations where RPCs are not unique

                    time_diffs_are_valid = True
                    for i in range(len(combo)):
                        for j in range(i + 1, len(combo)):
                            # error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                            error_window = 0
                            height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                            time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                            expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                            uncertainty_margin = 3

                            if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                                time_diffs_are_valid = False
                                break
                        if not time_diffs_are_valid:
                            break

                    if not time_diffs_are_valid:
                        continue  # Skip to the next combination if time differences are invalid

                    # Calculate metrics only for valid combinations
                    metric = calculate_metric_for_combo(combo, rpc_heights, direction)
                    if metric[2] <= 3.5:
                        valid_combinations.append(metric)
                        

                for valid_combination in valid_combinations:
                    paths.append({
                        'Event Number': event_number,
                        'Direction': direction,
                        'Slope': valid_combination[0],
                        'Intercept': valid_combination[1],
                        'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                        'chi2 norm': valid_combination[2],
                        'RSS': valid_combination[3]
                    })

    path_df = pd.DataFrame(paths)
    return path_df

In [374]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Chi2 Analysis Best here


In [433]:
Reconstructed_df_6 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 3)

Processing Events: 100%|██████████| 91796/91796 [03:27<00:00, 442.37it/s]


In [561]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [431]:
Temp = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 5)

Processing Events: 100%|██████████| 206130/206130 [01:28<00:00, 2319.54it/s]


In [232]:
df_no_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc3']

In [233]:
Reconstructed_df_6 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc3, 5)

Processing Events: 100%|██████████| 89822/89822 [00:26<00:00, 3326.75it/s]


In [48]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [92]:
Reconstructed_df_6.to_excel('Reconstructed_df_5.xlsx')

In [153]:
Reconstructed_df_3 = pd.read_excel('Reconstructed_df_3.xlsx')

In [444]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


KeyboardInterrupt: 

In [171]:
print(Reconstructed_df_6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6534 entries, 0 to 6533
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      6534 non-null   int64  
 1   Direction         6534 non-null   object 
 2   Slope             6534 non-null   float64
 3   Intercept         6534 non-null   float64
 4   Used Coordinates  6534 non-null   object 
 5   chi2 norm         6534 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 306.4+ KB
None


In [210]:
def parse_times(row):
    coords =row['Used Coordinates']
    times = {0: [], 1: [], 2: []}

    for coord in coords:
        rpc_id = int(coord[0][3])  
        time = coord[2]
        if rpc_id in [0, 1, 2]:
            times[0].append(time)
        elif rpc_id == 3:
            times[1].append(time)
        elif rpc_id in [4, 5]:
            times[2].append(time)

    for key in times:
        if times[key]:
            times[key] = sum(times[key]) / len(times[key])
        else:
            times[key] = None  

    return times

def determine_direction(times):
    t1, t2, t3 = times[0], times[1], times[2]

    if t1 is not None and t3 is not None:
        return 'bottom' if t1 < t3 else 'top'
    elif t1 is not None and t2 is not None:
        return 'bottom' if t1 < t2 else 'top'
    elif t2 is not None and t3 is not None:
        return 'bottom' if t2 < t3 else 'top'
    else:
        return 'unknown'  

Reconstructed_df_6['Parsed Times'] = Reconstructed_df_6.apply(parse_times, axis=1)
Reconstructed_df_6['Muon Direction'] = Reconstructed_df_6['Parsed Times'].apply(determine_direction)

In [212]:
Reconstructed_df_6 = Reconstructed_df_6[Reconstructed_df_6['Muon Direction'] == 'top']

In [213]:
show(Reconstructed_df_6.head())

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [434]:
def refine_reconstructed_paths(df):
    df['Number_of_Rpcs'] = df['Used Coordinates'].apply(len)

    # max_rpcs_df = df.groupby(['Event Number', 'Direction']).apply(
    #     lambda x: x[x['Number_of_Rpcs'] == x['Number_of_Rpcs'].max()]
    # ).reset_index(drop=True)

    # refined_df = max_rpcs_df.groupby(['Event Number', 'Direction']).apply(
    #     lambda x: x.nsmallest(1, 'chi2 norm')
    # ).reset_index(drop=True)
    refined_df = df.groupby(['Event Number', 'Direction']).apply(
        lambda x: x.nsmallest(3, 'chi2 norm')
    ).reset_index(drop=True)

    return refined_df

filtered_df = refine_reconstructed_paths(Reconstructed_df_6)

In [435]:
def refine_reconstructed_paths(df):
    df['Number_of_Rpcs'] = df['Used Coordinates'].apply(len)

    max_rpcs_df = df.groupby(['Event Number', 'Direction']).apply(
        lambda x: x[x['Number_of_Rpcs'] == x['Number_of_Rpcs'].max()]
    ).reset_index(drop=True)

    refined_df = max_rpcs_df.groupby(['Event Number', 'Direction']).apply(
        lambda x: x.nsmallest(3, 'chi2 norm')
    ).reset_index(drop=True)
    
    return refined_df

filtered_df = refine_reconstructed_paths(Reconstructed_df_6)

In [283]:
def check_rpc_conditions(coords):
    has_rpc3 = any('rpc3' == rpc for rpc, _, _ in coords)
    has_rpc4_or_rpc5 = any(rpc in ['rpc4', 'rpc5'] for rpc, _, _ in coords)
    return has_rpc3 and has_rpc4_or_rpc5

def extract_rpc_set(coords):
    return set(rpc for rpc, _, _ in coords)

# Function to filter events based on matching RPCs in 'eta' and 'phi'
def filter_matching_rpcs(group):
    if 'eta' in group['Direction'].values and 'phi' in group['Direction'].values:
        eta_rpcs = group[group['Direction'] == 'eta']['Used Coordinates'].apply(extract_rpc_set).iloc[0]
        phi_rpcs = group[group['Direction'] == 'phi']['Used Coordinates'].apply(extract_rpc_set).iloc[0]
        if eta_rpcs == phi_rpcs:
            return group
    return pd.DataFrame()  # Return empty DataFrame for non-matching or incomplete groups

# Group by 'Event Number' and apply the filter function
refined_df = filtered_df.groupby('Event Number').apply(filter_matching_rpcs).reset_index(drop=True)

In [264]:
show(filtered_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [437]:
# def refine_reconstructed_paths(df):
#     df['Number_of_Rpcs'] = df['Used Coordinates'].apply(len)

#     max_rpcs_df = df.groupby(['Event Number', 'Direction']).apply(
#         lambda x: x[x['Number_of_Rpcs'] == x['Number_of_Rpcs'].max()]
#     ).reset_index(drop=True)

#     refined_df = max_rpcs_df.groupby(['Event Number', 'Direction']).apply(
#         lambda x: x.nsmallest(1, 'chi2 norm')
#     ).reset_index(drop=True)

#     return refined_df

# refined_df = refine_reconstructed_paths(filtered_df)



In [175]:
print(refined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      956 non-null    float64
 1   Direction         956 non-null    object 
 2   Slope             956 non-null    float64
 3   Intercept         956 non-null    float64
 4   Used Coordinates  956 non-null    object 
 5   chi2 norm         956 non-null    float64
 6   Number_of_Rpcs    956 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 52.4+ KB
None


In [284]:
valid_events = refined_df.groupby('Event Number')['Direction'].agg(set).reset_index()
valid_events = valid_events[valid_events['Direction'] == {'eta', 'phi'}]

# Filter the original DataFrame to include only those Event Numbers with both directions
filtered_refined_df = refined_df[refined_df['Event Number'].isin(valid_events['Event Number'])]


In [245]:
show(filtered_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [423]:
print(filtered_refined_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98476 entries, 0 to 98475
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      98476 non-null  float64
 1   Direction         98476 non-null  object 
 2   Slope             98476 non-null  float64
 3   Intercept         98476 non-null  float64
 4   Used Coordinates  98476 non-null  object 
 5   chi2 norm         98476 non-null  float64
 6   RSS               98476 non-null  object 
 7   Number_of_Rpcs    98476 non-null  int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 6.8+ MB
None


In [164]:
show(filtered_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [157]:
filtered_refined_df.to_csv('test')

In [148]:
print(filtered_refined_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582 entries, 0 to 581
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      582 non-null    float64
 1   Direction         582 non-null    object 
 2   Slope             582 non-null    float64
 3   Intercept         582 non-null    float64
 4   Used Coordinates  582 non-null    object 
 5   chi2 norm         582 non-null    float64
 6   Number_of_Rpcs    582 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 36.4+ KB
None


In [56]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [ ]:
show(filtered_df)

In [295]:
show(filtered_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [297]:
def is_continuous(rpc_list):
    # Extracting the index number from each tuple ('rpcX')
    indices = [int(rpc[0][3:]) for rpc in rpc_list]  # 'rpcX' -> X
    # Check if sorted indices are continuous
    sorted_indices = sorted(indices)
    return sorted_indices == list(range(min(sorted_indices), max(sorted_indices)+1))

# Applying the filter to the DataFrame
filtered_df['is_continuous'] = filtered_df['Used Coordinates'].apply(is_continuous)

# Filtering the DataFrame to keep only continuous rows
filtered_refined_contineous_df = filtered_df[filtered_df['is_continuous']]

# Drop the auxiliary column
filtered_refined_contineous_df.drop(columns='is_continuous', inplace=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_25604\1105133516.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [301]:
rpc_heights = {
    'rpc0': 0, 
    'rpc1': 0.5, 
    'rpc2': 1.0, 
    'rpc3': 61.5, 
    'rpc4': 121.5, 
    'rpc5': 122.0
}
    
def should_delete_row(row):
    used_rpcs = [rpc[0] for rpc in row['Used Coordinates']]
    slope = row['Slope']
    intercept = row['Intercept']
    direction = row['Direction']
    
    # Define bounds based on direction
    if direction == 'eta':
        bounds = (-32 * 2.5,0)
    elif direction == 'phi':
        bounds = (0, 64 * 2.5)
    else:
        raise ValueError("Unknown direction")

    # Calculate X for all rpc_heights and check if any outside used_rpcs lie within bounds
    for rpc, height in rpc_heights.items():
        if rpc not in used_rpcs:
            x_value = slope * height + intercept
            if bounds[0] <= x_value <= bounds[1]:
                return True  # Indicates row should be deleted

    return False

# Apply the function and filter rows
filtered_refined_contineous_df['Delete Row'] = filtered_refined_contineous_df.apply(should_delete_row, axis=1)
filtered_refined_contineous_Deleted_df = filtered_refined_contineous_df[~filtered_refined_contineous_df['Delete Row']]  # Keep rows where 'Delete Row' is False

C:\Users\Peter\AppData\Local\Temp\ipykernel_25604\2894165069.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
show(fiultered_df)

In [416]:
Reconstructed_df_6_cleaned = Reconstructed_df_6

This one best one

In [436]:
Reconstructed_df_6_cleaned = filtered_df[filtered_df['chi2 norm'] > 0.005]
#BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST

In [362]:
Reconstructed_df_6_cleaned = filtered_df

In [312]:
Reconstructed_df_6_cleaned = filtered_refined_df[filtered_refined_df['chi2 norm'] != 0]

In [307]:
Reconstructed_df_6_cleaned = filtered_refined_contineous_df

In [302]:
Reconstructed_df_6_cleaned = filtered_refined_contineous_Deleted_df

In [ ]:
show(filtered_df)

In [437]:
eta_df = Reconstructed_df_6_cleaned[Reconstructed_df_6_cleaned['Direction'] == 'eta']
angles_y_axis = np.degrees(np.arctan(-eta_df['Slope']))

angles_y_axis = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)
angles_y_axis = np.where(angles_y_axis < -90, angles_y_axis + 180, angles_y_axis)


plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=30, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Any threeAngular_Distribution_eta_real_filtered')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [388]:
eta_df = Reconstructed_df_6_cleaned[Reconstructed_df_6_cleaned['Direction'] == 'phi']
angles_y_axis = np.degrees(np.arctan(eta_df['Slope']))

# angles_y_axis = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)
# angles_y_axis = np.where(angles_y_axis < -90, angles_y_axis + 180, angles_y_axis)


plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=50, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Any threeAngular_Distribution_phi_real_filtered')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [569]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [253]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = Reconstructed_df_6[Reconstructed_df_6['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=600, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 5 Atlas Luminous generous window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = Reconstructed_df_6[Reconstructed_df_6['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 phi at least 5 Atlas Luminous generous window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


In [49]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = refined_df[refined_df['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=50, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 5 Atlas Luminous generous window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = refined_df[refined_df['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=50, color='red', alpha=0.7)
plt.title('Chi2 phi at least 5 Atlas Luminous generous window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()

In [50]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = filtered_refined_df[filtered_refined_df['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=100, color='green', alpha=0.7)
plt.title('Chi2 Eta At least 5Cross entire Volumne Atlas Luminous')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = filtered_refined_df[filtered_refined_df['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=100, color='red', alpha=0.7)
plt.title('Chi2 phi At least 5 Cross Entire Volume Atlas Luminous')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


In [250]:
filtered_refined_df['mean_RSS'] = pd.NA

var = 2.5 ** 2 / 12

# Calculate mean RSS for each group and assign it back to the original DataFrame
for event_number, group in filtered_refined_df.groupby('Event Number'):
    chi2_together = (group['RSS'].sum() / var) / (group['Number_of_Rpcs'] - 3)
    filtered_refined_df.loc[filtered_refined_df['Event Number'] == event_number, 'Together chi 2'] = chi2_together

In [253]:
# Setting up the figure
plt.figure(figsize=(24, 12))

eta_chi2 = filtered_refined_df[filtered_refined_df['Direction'] == 'eta']['Together chi 2']
plt.hist(eta_chi2, bins=100, color='green', alpha=0.7)
plt.title('Chi2 3D together')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)
# Display the plots
plt.tight_layout()
plt.show()


Timing for triplet

In [262]:
show(filtered_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [440]:
eta_refined_df = filtered_refined_df[filtered_refined_df['Direction'] == 'eta']

In [286]:
print(times)

[('rpc1', -25, 249.0), ('rpc1', -14, 278.0), ('rpc1', -24, 259.0), ('rpc1', -22, 254.5), ('rpc1', -17, 256.0), ('rpc1', -9, 255.5), ('rpc1', -30, 248.5), ('rpc1', -23, 248.0), ('rpc1', -8, 249.5), ('rpc1', -24, 266.0), ('rpc1', -4, 252.5), ('rpc1', -14, 252.0), ('rpc1', -5, 256.0), ('rpc1', -24, 249.0), ('rpc1', -21, 255.0), ('rpc1', -10, 245.0), ('rpc1', -18, 252.0), ('rpc1', -6, 249.0), ('rpc1', -29, 249.0), ('rpc1', -7, 251.5), ('rpc1', -5, 252.0), ('rpc1', -26, 268.5), ('rpc1', -5, 251.0), ('rpc1', -11, 253.0), ('rpc1', -15, 246.0), ('rpc1', -9, 257.0), ('rpc1', -15, 249.0), ('rpc1', -15, 269.0), ('rpc1', -8, 251.0), ('rpc1', -12, 250.0), ('rpc1', -15, 256.0), ('rpc1', -7, 261.0), ('rpc1', -13, 250.0), ('rpc1', -15, 273.5), ('rpc1', -4, 256.0), ('rpc1', -31, 241.0), ('rpc1', -18, 267.0), ('rpc1', -15, 254.5), ('rpc1', -25, 249.0), ('rpc1', -25, 250.5), ('rpc1', -12, 251.0), ('rpc1', -12, 250.0), ('rpc1', -10, 254.0), ('rpc1', -24, 240.0), ('rpc1', -23, 249.0), ('rpc1', -8, 251.5), 

In [441]:
def find_triplet_time_range(rpc_list):
    # Extract rpc names, time, and data into separate lists
    rpc_names = [x[0] for x in rpc_list]
    times = {x[0]: x[2] for x in rpc_list}
    
    # Check if 'rpc0', 'rpc1', 'rpc2' exist in the names
    required_rpcs = ['rpc0', 'rpc1', 'rpc2']
    if all(rpc in rpc_names for rpc in required_rpcs):
        # Calculate the range of time and mean time for these RPCs
        time_values = [times[rpc] for rpc in required_rpcs]
        time_range = max(time_values) - min(time_values)
        mean_time = sum(time_values) / len(time_values)
        return (True, time_range, mean_time)
    else:
        return (False, None, None)
    
eta_refined_df['triplet range'] = eta_refined_df['Used Coordinates'].apply(find_triplet_time_range)

C:\Users\Peter\AppData\Local\Temp\ipykernel_32216\4050136835.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [355]:
show(eta_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [442]:
times = []
for k in eta_refined_df['triplet range']:
    times.append(k[1])
times = [x for x in times if x is not None]

print(times)

[1.0, 2.0, 2.0, 2.0, 3.0, 1.5, 2.0, 0.5, 0.5, 1.0, 2.0, 0.0, 2.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 4.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 3.0, 3.0, 2.0, 0.0, 1.0, 5.0, 1.0, 5.5, 0.5, 0.0, 1.0, 2.0, 3.0, 3.3333333333333144, 1.0, 3.5, 2.5, 2.3333333333333144, 1.5, 2.0, 1.0, 3.0, 1.0, 3.5, 1.0, 0.0, 0.0, 2.0, 3.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 3.0, 1.0, 2.0, 3.0, 0.0, 0.0, 7.5, 2.666666666666657, 1.0, 2.0, 0.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.5, 3.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 2.0, 3.5, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.5, 4.0, 2.0, 3.5, 1.0, 2.0, 2.5, 1.0, 0.0, 1.5, 3.0, 1.6666666666666572, 5.0, 1.0, 1.0, 1.5, 2.0, 1.0, 3.0, 2.0, 2.0, 1.0, 3.0, 4.0, 2.0, 1.0, 1.0, 2.5, 1.0, 1.0, 0.0, 1.0, 0.0, 3.0, 2.0, 0.0, 2.0, 2.0, 2.75, 2.0, 2.0, 1.0, 1.0, 3.0, 1.5, 2.0, 1.0, 2.0, 1.5, 2.5, 3.5, 3.0, 1.5, 1.0, 2.0, 3.0, 2.0, 1.0, 2.5, 2.0, 1.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 8.0, 3.0, 4.0, 1.5, 5.333333333333314, 1.0, 2.0, 1.0, 1.0, 1.0, 1.5, 1.0, 2.0, 1.0, 2.25, 3.0,

In [443]:
plt.figure(figsize=(12, 6))

Triplet_range = eta_refined_df['triplet range']
plt.hist(times, bins=10, color='green', alpha=0.7)
plt.title('Triplet timing range for at least 5 rpc muons eta')
plt.xlabel('range/ns')
plt.ylabel('Number')

plt.grid(True)
# Display the plots
plt.tight_layout()
plt.show()


In [364]:
event_time_triplet = []
for k in eta_refined_df['triplet range']:
    event_time_triplet.append(k[2])
event_time_triplet = [x for x in event_time_triplet if x is not None]

print(event_time_triplet)

[256.3333333333333, 249.33333333333334, 253.0, 251.0, 270.3333333333333, 249.16666666666666, 266.0, 259.1666666666667, 243.16666666666666, 249.66666666666666, 248.66666666666666, 260.0, 246.33333333333334, 253.66666666666666, 243.66666666666666, 250.33333333333334, 249.33333333333334, 249.33333333333334, 244.66666666666666, 252.83333333333334, 251.5, 262.0, 243.0, 271.8333333333333, 266.0, 248.66666666666666, 254.66666666666666, 250.66666666666666, 247.0, 247.0, 255.0, 263.0, 272.3333333333333, 245.5, 254.16666666666666, 257.0, 250.66666666666666, 248.66666666666666, 258.3333333333333, 270.4444444444444, 270.3333333333333, 247.83333333333334, 254.83333333333334, 266.1111111111111, 254.0, 272.3333333333333, 251.33333333333334, 251.33333333333334, 247.33333333333334, 250.5, 251.33333333333334, 252.0, 253.0, 247.0, 262.3333333333333, 273.3333333333333, 254.33333333333334, 256.3333333333333, 254.66666666666666, 243.66666666666666, 262.3333333333333, 248.33333333333334, 256.3333333333333, 2

In [365]:
plt.figure(figsize=(12, 6))

plt.hist(event_time_triplet, bins=20, color='green', alpha=0.7)
plt.title('Triplet event time for all 6 rpc muons eta')
plt.xlabel('range/ns')
plt.ylabel('Number')

plt.grid(True)
# Display the plots
plt.tight_layout()
plt.show()


In [268]:
show(eta_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Cluster timing

In [367]:
show(cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [380]:
Triplet_cluster_df = cluster_df[cluster_df['rpc_number'].isin(['rpc0', 'rpc1', 'rpc2'])]

In [370]:
show(Triplet_cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [371]:
print(Triplet_cluster_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580577 entries, 0 to 759898
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     580577 non-null  int64 
 1   rpc_number       580577 non-null  object
 2   strip_direction  580577 non-null  object
 3   locations        580577 non-null  object
 4   times            580577 non-null  object
 5   start_time       580577 non-null  int64 
 6   end_time         580577 non-null  int64 
 7   size             580577 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 39.9+ MB
None


In [392]:
Triplet_cluster_df_filtered = Triplet_cluster_df[Triplet_cluster_df['size'] > 1]

# Compute max(times) - min(times) for each row
Triplet_cluster_df_filtered['time_diff'] = Triplet_cluster_df_filtered['times'].apply(lambda x: max(x) - min(x))

# Filter data based on rpc_number
rpc0_data = Triplet_cluster_df_filtered[Triplet_cluster_df_filtered['rpc_number'] == 'rpc0']
rpc1_data = Triplet_cluster_df_filtered[Triplet_cluster_df_filtered['rpc_number'] == 'rpc1']
rpc2_data = Triplet_cluster_df_filtered[Triplet_cluster_df_filtered['rpc_number'] == 'rpc2']

# Plotting the histograms
plt.figure(figsize=(12, 8))
plt.hist(rpc0_data['time_diff'], bins=20, alpha=0.7, color='red', edgecolor='red', label='RPC0', histtype='step', linewidth=2)
plt.hist(rpc1_data['time_diff'], bins=20, alpha=0.7, color='green', edgecolor='green', label='RPC1', histtype='step', linewidth=2)
plt.hist(rpc2_data['time_diff'], bins=20, alpha=0.7, color='blue', edgecolor='blue', label='RPC2', histtype='step', linewidth=2)

plt.title('Cluster Max-Min Times Difference in Triplet for Each RPC Number')
plt.xlabel('Time Difference (nanoseconds)')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()

C:\Users\Peter\AppData\Local\Temp\ipykernel_32216\2565581196.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [407]:
# Assuming Triplet_cluster_df is already defined and loaded as per your setup
Triplet_cluster_df_filtered = Triplet_cluster_df[Triplet_cluster_df['size'] > 1]

# Compute max(times) - min(times) for each row
Triplet_cluster_df_filtered['time_diff'] = Triplet_cluster_df_filtered['times'].apply(lambda x: max(x) - min(x))

# Prepare the plot
plt.figure(figsize=(12, 8))

# Colors and styles for visual distinction
colors = ['red', 'green', 'blue']
styles = ['-', '--', ':']
rpc_numbers = Triplet_cluster_df_filtered['rpc_number'].unique()
sizes = Triplet_cluster_df_filtered['size'].unique()


for size_index, size in enumerate(sorted(sizes)):
    if size <=5:
        # Filter data for each rpc_number and size
        data = Triplet_cluster_df_filtered[
            (Triplet_cluster_df_filtered['size'] == size)
        ]

        # Only plot if there is data
        if not data.empty:
            plt.hist(data['time_diff'], bins=15, alpha=0.7, label=f'Size {size}')

# Plot customization
plt.title('Cluster Max-Min Times Difference in Triplet for Each RPC Number and Size')
plt.xlabel('Time Difference (nanoseconds)')
plt.ylabel('Frequency')
plt.grid(True)
plt.yscale('log')
plt.legend(title='RPC and Size')
plt.show()

C:\Users\Peter\AppData\Local\Temp\ipykernel_32216\468951302.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [375]:
show(Triplet_cluster_df_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [137]:
print(Reconstructed_df_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88254 entries, 0 to 88253
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      88254 non-null  int64  
 1   Direction         88254 non-null  object 
 2   Slope             88254 non-null  float64
 3   Intercept         88254 non-null  float64
 4   Used Coordinates  88254 non-null  object 
 5   chi2 norm         88254 non-null  float64
 6   Number_of_Rpcs    88254 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 4.7+ MB
None


In [167]:
show(Reconstructed_df_4)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [190]:
show(Reconstructed_df_3)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [175]:
print(Reconstructed_df_4['Used Coordinates'])

0        [(rpc0, 51, 250.04666666666665), (rpc1, 51, 24...
1        [(rpc0, 22, 237.38), (rpc1, 22, 237.79), (rpc2...
2        [(rpc0, 49, 244.38), (rpc1, 49, 243.79), (rpc2...
3        [(rpc0, 49, 244.38), (rpc1, 49, 243.79), (rpc2...
4        [(rpc0, 49, 244.38), (rpc1, 49, 243.79), (rpc2...
                               ...                        
88249    [(rpc0, 3, 231.38), (rpc1, 3, 230.79), (rpc2, ...
88250    [(rpc0, 3, 231.38), (rpc1, 3, 230.79), (rpc2, ...
88251    [(rpc0, 56, 280.38), (rpc1, 56, 269.79), (rpc2...
88252    [(rpc0, -27, 255.06), (rpc1, -27, 256.64), (rp...
88253    [(rpc0, -27, 255.06), (rpc1, -27, 256.64), (rp...
Name: Used Coordinates, Length: 88254, dtype: object


In [186]:
def convert_string_to_tuples(data):
    try:
        # Convert the string representation back to the actual data type (list of tuples)
        return ast.literal_eval(data)
    except (SyntaxError, ValueError) as e:
        # Handle errors that arise if the data is not in the expected format
        print(f"Error converting data: {data}. Error: {e}")
        return None
    
Reconstructed_df_3['Used Coordinates'] = Reconstructed_df_3['Used Coordinates'].apply(convert_string_to_tuples)

In [196]:
def check_rpc_conditions(coords):
    has_rpc3 = any('rpc3' == rpc for rpc, _, _ in coords)
    has_rpc4_or_rpc5 = any(rpc in ['rpc4', 'rpc5'] for rpc, _, _ in coords)
    return has_rpc3 and has_rpc4_or_rpc5

def extract_rpc_set(coords):
    return set(rpc for rpc, _, _ in coords)

# Function to filter events based on matching RPCs in 'eta' and 'phi'
def filter_matching_rpcs(group):
    if 'eta' in group['Direction'].values and 'phi' in group['Direction'].values:
        eta_rpcs = group[group['Direction'] == 'eta']['Used Coordinates'].apply(extract_rpc_set).iloc[0]
        phi_rpcs = group[group['Direction'] == 'phi']['Used Coordinates'].apply(extract_rpc_set).iloc[0]
        if eta_rpcs == phi_rpcs:
            return group
    return pd.DataFrame()  # Return empty DataFrame for non-matching or incomplete groups

# Group by 'Event Number' and apply the filter function
Reconstructed_df_3 = Reconstructed_df_3.groupby('Event Number').apply(filter_matching_rpcs).reset_index(drop=True)



# Apply the function to filter the dataframe
Cross_condition_df = Reconstructed_df_3[Reconstructed_df_3['Used Coordinates'].apply(check_rpc_conditions)]
Cross_condition_df = refine_reconstructed_paths(Cross_condition_df)
valid_events = Cross_condition_df.groupby('Event Number')['Direction'].agg(set).reset_index()
valid_events = valid_events[valid_events['Direction'] == {'eta', 'phi'}]

# Filter the original DataFrame to include only those Event Numbers with both directions
Cross_condition_df = Cross_condition_df[Cross_condition_df['Event Number'].isin(valid_events['Event Number'])]

C:\Users\Peter\AppData\Local\Temp\ipykernel_40476\1569048960.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [200]:
show(Reconstructed_df_3)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [202]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = Cross_condition_df[Cross_condition_df['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=60, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = Cross_condition_df[Cross_condition_df['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 Phi at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()

In [201]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = Reconstructed_df_3[Reconstructed_df_3['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=60, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = Reconstructed_df_3[Reconstructed_df_3['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 Phi at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()

In [87]:
Reconstructed_df_6 = Reconstructed_df_6.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
              .reset_index(drop=True)

In [23]:
print(Reconstructed_df_6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      964 non-null    int64  
 1   Direction         964 non-null    object 
 2   Slope             964 non-null    float64
 3   Intercept         964 non-null    float64
 4   Slope_error       964 non-null    int64  
 5   Intercept_error   964 non-null    int64  
 6   Used Coordinates  964 non-null    object 
 7   RSS               964 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 60.4+ KB
None


In [59]:
print(len([1,2,3,4]))

4


In [22]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [49]:
var_eta =  ( 1 * 3.09375) ** 2 / 12
var_phi = ( 1 * 2.8125) ** 2 / 12
rpc_heights = {
    'rpc0': 0, 
    'rpc1': 0.5, 
    'rpc2': 1.0, 
    'rpc3': 61.5, 
    'rpc4': 121.5, 
    'rpc5': 122.0
}

In [109]:
# Constants for scaling heights (since y values, which are heights, now determine x)
scale_eta = 3.09375
scale_phi = 2.8125

# Function to parse used coordinates, scale y based on direction and extract x values as dependent variables
def parse_and_scale_coordinates(used_coords, direction):
    if isinstance(used_coords, str):
        coords = ast.literal_eval(used_coords)
    else:
        coords = used_coords
    
    # Determine the scale factor based on the direction
    scale_factor = scale_eta if direction == 'eta' else scale_phi

    # Extract x and y values, scale y values
    y_vals = [rpc_heights[coord[0]] for coord in coords]
    x_vals = [coord[1] * scale_factor for coord in coords]
    return np.array(x_vals), np.array(y_vals)

# Apply the parsing and scaling function to each row based on direction
Reconstructed_df_6[['x', 'y']] = Reconstructed_df_6.apply(
    lambda row: pd.Series(parse_and_scale_coordinates(row['Used Coordinates'], row['Direction']), index=['x', 'y']),
    axis=1
)

# Function to calculate chi-squared
def calculate_chi_squared(row):
    x = np.array(row['x'])
    y = np.array(row['y'])
    a = row['Slope']
    b = row['Intercept']
    x_pred = a * y + b  # New model

    # Determine variance based on direction
    if row['Direction'] == 'eta':
        sigma_squared = var_eta
    elif row['Direction'] == 'phi':
        sigma_squared = var_phi
    else:
        sigma_squared = 1  # Default case, adjust as necessary

    chi_squared = np.sum(((x - x_pred) ** 2) / sigma_squared)
    chi_squared_norm = chi_squared / (row['Number_of_Rpcs'] - 2)
    return chi_squared_norm

# Apply chi-squared calculation to each row
Reconstructed_df_6['Chi2'] = Reconstructed_df_6.apply(calculate_chi_squared, axis=1)


In [110]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [219]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = Reconstructed_df_6[Reconstructed_df_6['Direction'] == 'eta']['Chi2']
plt.hist(eta_chi2, bins=60, color='green', alpha=0.7)
plt.title('Chi2 Eta all 6 Atlas Luminous')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')
plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = Reconstructed_df_6[Reconstructed_df_6['Direction'] == 'phi']['Chi2']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 phi all 6 Atlas Luminous')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')
plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


KeyError: 'Chi2'

In [218]:

# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = refined_df[refined_df['Direction'] == 'eta']['Chi2']
plt.hist(eta_chi2, bins=60, color='green', alpha=0.7)
plt.title('Chi2 Eta all 6 Atlas Luminous')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')
plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = refined_df[refined_df['Direction'] == 'phi']['Chi2']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 phi all 6 Atlas Luminous')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')
plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


KeyError: 'Chi2'

In [128]:
df_no_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc0']
df_no_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc1']
df_no_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc2']
df_no_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc3']
df_no_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc4']
df_no_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc5']


In [ ]:
df_recon_no_rpc0_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc0, 5)
df_recon_no_rpc1_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc1, 5)
df_recon_no_rpc2_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc2, 5)
df_recon_no_rpc3_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc3, 5)
df_recon_no_rpc4_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc4, 5)
df_recon_no_rpc5_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc5, 5)

Processing Events: 100%|██████████| 92919/92919 [01:17<00:00, 1196.73it/s]


In [ ]:
df_recon_no_rpc0_4.to_excel('df_recon_no_rpc0_5_AtlasOFF.xlsx')
df_recon_no_rpc1_4.to_excel('df_recon_no_rpc1_5_AtlasOFF.xlsx')
df_recon_no_rpc2_4.to_excel('df_recon_no_rpc2_5_AtlasOFF.xlsx')
df_recon_no_rpc3_4.to_excel('df_recon_no_rpc3_5_AtlasOFF.xlsx')
df_recon_no_rpc4_4.to_excel('df_recon_no_rpc4_5_AtlasOFF.xlsx')
df_recon_no_rpc5_4.to_excel('df_recon_no_rpc5_5_AtlasOFF.xlsx')

In [129]:
df_recon_no_rpc0_4 = pd.read_excel('df_recon_no_rpc0_5_AtlasON.xlsx')
df_recon_no_rpc1_4 = pd.read_excel('df_recon_no_rpc1_5_AtlasON.xlsx')
df_recon_no_rpc2_4 = pd.read_excel('df_recon_no_rpc2_5_AtlasON.xlsx')
df_recon_no_rpc3_4 = pd.read_excel('df_recon_no_rpc3_5_AtlasON.xlsx')
df_recon_no_rpc4_4 = pd.read_excel('df_recon_no_rpc4_5_AtlasON.xlsx')
df_recon_no_rpc5_4 = pd.read_excel('df_recon_no_rpc5_5_AtlasON.xlsx')

In [129]:
print(df_recon_no_rpc0_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      1253 non-null   int64  
 1   Direction         1253 non-null   object 
 2   Slope             1253 non-null   float64
 3   Intercept         1253 non-null   float64
 4   Slope_error       1253 non-null   int64  
 5   Intercept_error   1253 non-null   int64  
 6   Used Coordinates  1253 non-null   object 
 7   RSS               1253 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 78.4+ KB
None


In [75]:
show(df_recon_no_rpc0_4.sort_values('Event Number'))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [130]:
df_recon_no_rpc0_4_filtered = df_recon_no_rpc0_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
              .reset_index(drop=True)
              
df_recon_no_rpc1_4_filtered = df_recon_no_rpc1_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
                                .reset_index(drop=True)
df_recon_no_rpc2_4_filtered = df_recon_no_rpc2_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
              
df_recon_no_rpc3_4_filtered = df_recon_no_rpc3_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
df_recon_no_rpc4_4_filtered = df_recon_no_rpc4_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
df_recon_no_rpc5_4_filtered = df_recon_no_rpc5_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)

In [74]:
show(df_recon_no_rpc0_4_filtered.reset_index(drop=True))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [131]:
df_recon_no_rpc0_4_filtered['expected hits'] = df_recon_no_rpc0_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc1_4_filtered['expected hits'] = df_recon_no_rpc1_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc2_4_filtered['expected hits'] = df_recon_no_rpc2_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc3_4_filtered['expected hits'] = df_recon_no_rpc3_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc4_4_filtered['expected hits'] = df_recon_no_rpc4_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc5_4_filtered['expected hits'] = df_recon_no_rpc5_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)

In [132]:
fully_massaged_df_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc0']
fully_massaged_df_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc1']
fully_massaged_df_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc2']
fully_massaged_df_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc3']
fully_massaged_df_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc4']
fully_massaged_df_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc5']

def calculate_midpoint(locations, direction):
    if direction == 'eta':
        return np.mean([loc[1] for loc in locations])
    elif direction == 'phi':
        return np.mean([loc[0] for loc in locations])
    return None

fully_massaged_df_rpc0['midpoint'] = fully_massaged_df_rpc0.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc1['midpoint'] = fully_massaged_df_rpc1.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc2['midpoint'] = fully_massaged_df_rpc2.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc3['midpoint'] = fully_massaged_df_rpc3.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc4['midpoint'] = fully_massaged_df_rpc4.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc5['midpoint'] = fully_massaged_df_rpc5.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)

merged_df0 = pd.merge(df_recon_no_rpc0_4_filtered, fully_massaged_df_rpc0, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df1 = pd.merge(df_recon_no_rpc1_4_filtered, fully_massaged_df_rpc1, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df2 = pd.merge(df_recon_no_rpc2_4_filtered, fully_massaged_df_rpc2, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df3 = pd.merge(df_recon_no_rpc3_4_filtered, fully_massaged_df_rpc3, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df4 = pd.merge(df_recon_no_rpc4_4_filtered, fully_massaged_df_rpc4, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df5 = pd.merge(df_recon_no_rpc5_4_filtered, fully_massaged_df_rpc5, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')


merged_df0['hit_difference'] = merged_df0['expected hits'] - merged_df0['midpoint']
merged_df1['hit_difference'] = merged_df1['expected hits'] - merged_df1['midpoint']
merged_df2['hit_difference'] = merged_df2['expected hits'] - merged_df2['midpoint']
merged_df3['hit_difference'] = merged_df3['expected hits'] - merged_df3['midpoint']
merged_df4['hit_difference'] = merged_df4['expected hits'] - merged_df4['midpoint']
merged_df5['hit_difference'] = merged_df5['expected hits'] - merged_df5['midpoint']

result_df0 = merged_df0[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df1 = merged_df1[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df2 = merged_df2[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df3 = merged_df3[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df4 = merged_df4[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df5 = merged_df5[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]


C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [133]:
eta_df0 = result_df0[result_df0['Direction'] == 'eta']
eta_df1 = result_df1[result_df1['Direction'] == 'eta']
eta_df2 = result_df2[result_df2['Direction'] == 'eta']
eta_df3 = result_df3[result_df3['Direction'] == 'eta']
eta_df4 = result_df4[result_df4['Direction'] == 'eta']
eta_df5 = result_df5[result_df5['Direction'] == 'eta']

phi_df0 = result_df0[result_df0['Direction'] == 'phi']
phi_df1 = result_df1[result_df1['Direction'] == 'phi']
phi_df2 = result_df2[result_df2['Direction'] == 'phi']
phi_df3 = result_df3[result_df3['Direction'] == 'phi']
phi_df4 = result_df4[result_df4['Direction'] == 'phi']
phi_df5 = result_df5[result_df5['Direction'] == 'phi']

In [134]:
def get_min_index(group):
    if group['hit_difference'].isna().all():
        return group.index.tolist() 
    return [group['hit_difference'].idxmin()]


In [135]:
phi_df0 = phi_df0.loc[phi_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df1 = phi_df1.loc[phi_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df2 = phi_df2.loc[phi_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df3 = phi_df3.loc[phi_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df4 = phi_df4.loc[phi_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df5 = phi_df5.loc[phi_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]

eta_df0 = eta_df0.loc[eta_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df1 = eta_df1.loc[eta_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df2 = eta_df2.loc[eta_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df3 = eta_df3.loc[eta_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df4 = eta_df4.loc[eta_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df5 = eta_df5.loc[eta_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]



In [138]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction RPC0 any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [118]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction RPC0 any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [119]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(6):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(6):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        total_valid = combined_df[i].shape[0]
        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

In [31]:
show(combined_df[0])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [32]:
show(eta_df0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Chi2 Analysis

In [149]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
df_chi2_unfiltered = [df_recon_no_rpc0_4, df_recon_no_rpc1_4, df_recon_no_rpc2_4, df_recon_no_rpc3_4, df_recon_no_rpc4_4, df_recon_no_rpc5_4]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(len(df_chi2)):
    df_chi2[i]['Chi_squared'] = df_chi2[i]['RSS'] / 1.333

plt.figure(figsize=(10, 6))
for i in range(len(df_chi2)):
    plt.hist(df_chi2[i]['Chi_squared'], bins=500, alpha=0.75, 
             label=label_rpc[i], histtype='step', linewidth=1.3, log = True)
plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Atlas Off')
plt.ylabel('Number of Occurance')
plt.legend()
plt.grid(True)
plt.show()

In [177]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
df_chi2_unfiltered = [df_recon_no_rpc0_4, df_recon_no_rpc1_4, df_recon_no_rpc2_4, df_recon_no_rpc3_4, df_recon_no_rpc4_4, df_recon_no_rpc5_4]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(len(df_chi2)):
    df_chi2[i]['Chi_squared'] = df_chi2[i]['RSS'] / 1

plt.figure(figsize=(10, 6))
for i in range(len(df_chi2)):
    plt.hist(df_chi2[i]['Chi_squared'], bins=500, alpha=0.75, 
             label=label_rpc[i], log = True)
plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Atlas ON')
plt.ylabel('Number of Occurance')
plt.legend()
plt.grid(True)
plt.show()

In [155]:
print(df_recon_no_rpc0_4_filtered.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1132 non-null   int64  
 1   Event Number      1132 non-null   int64  
 2   Direction         1132 non-null   object 
 3   Slope             1132 non-null   float64
 4   Intercept         1132 non-null   float64
 5   Slope_error       1132 non-null   int64  
 6   Intercept_error   1132 non-null   int64  
 7   Used Coordinates  1132 non-null   object 
 8   RSS               1132 non-null   float64
 9   Chi_squared       1132 non-null   float64
 10  expected hits     1132 non-null   float64
dtypes: float64(5), int64(4), object(2)
memory usage: 97.4+ KB
None


In [12]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']
df_eta_list = []
df_phi_list = []
                    
Variance_eta = ( 0.5 * 3.09375) ** 2 / 12
Variance_phi = ( 0.5 * 2.8125) ** 2 / 12

for df in df_chi2:
    df_eta = df[df['Direction'] == 'eta']
    df_phi = df[df['Direction'] == 'phi']
    
    # Append to lists
    df_eta_list.append(df_eta)
    df_phi_list.append(df_phi)

# Calculate Chi_squared values
for df in df_eta_list:
    df['Chi_squared'] = df['RSS'] / Variance_eta 
for df in df_phi_list:
    df['Chi_squared'] = df['RSS'] / Variance_phi 

plt.figure(figsize=(10, 6))
for i, df in enumerate(df_eta_list):
    plt.hist(df['Chi_squared'], bins=500, alpha=0.75, label=label_rpc[i], log = True)


plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Eta Atlas Off')
plt.ylabel('Number of Occurrence')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
for i, df in enumerate(df_phi_list):
    plt.hist(df['Chi_squared'], bins=500, alpha=0.75, label=label_rpc[i], log=True)

plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Phi Atlas Off')
plt.ylabel('Number of Occurrence')
plt.legend()
plt.grid(True)
plt.show()

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\220974511.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chi_squared'] = df['RSS'] / Variance_eta  # This is the same as just using RSS directly
C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\220974511.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chi_squared'] = df['RSS'] / Variance_phi  # This is the same as just using RSS directly


In [187]:
show(df_eta_list[0])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [151]:
show(df_recon_no_rpc0_4_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [13]:
show(eta_df0)

NameError: name 'eta_df0' is not defined

In [34]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]


efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(3):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [37]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(3):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [38]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(3):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(3):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        total_valid = combined_df[i].shape[0]

        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

Separate them into TDCs Using a reverse Mapping

In [43]:
show(phi_df1)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [136]:
reversed_mapping = {
    ('rpc0', 'eta'): [(0, 32, 0)],
    ('rpc0', 'phi'): [(0, 64, 0)],
    ('rpc1', 'eta'): [(0, 32, 0)],
    ('rpc1', 'phi'): [(0, 64, 1)],
    ('rpc2', 'eta'): [(0, 32, 1)],
    ('rpc2', 'phi'): [(0, 64, 1)],
    ('rpc3', 'eta'): [(0, 32, 2)],
    ('rpc3', 'phi'): [(0, 64, 2)],
    ('rpc4', 'eta'): [(0, 32, 3)],
    ('rpc4', 'phi'): [(0, 64, 3)],
    ('rpc5', 'eta'): [(0, 32, 3)],
    ('rpc5', 'phi'): [(0, 64, 4)]
}

for key, value in reversed_mapping.items():
    print(key, ":", value)
    
def compute_tdc_number(rpc, direction, expected_hits, mapping):
    if (rpc, direction) in mapping:
        for (start, end, chip) in mapping[(rpc, direction)]:  

            if start <= expected_hits <= end:

                return int(chip)
    return None  

print(compute_tdc_number('rpc1', 'eta', 12.021, reversed_mapping))

('rpc0', 'eta') : [(0, 32, 0)]
('rpc0', 'phi') : [(0, 64, 0)]
('rpc1', 'eta') : [(0, 32, 0)]
('rpc1', 'phi') : [(0, 64, 1)]
('rpc2', 'eta') : [(0, 32, 1)]
('rpc2', 'phi') : [(0, 64, 1)]
('rpc3', 'eta') : [(0, 32, 2)]
('rpc3', 'phi') : [(0, 64, 2)]
('rpc4', 'eta') : [(0, 32, 3)]
('rpc4', 'phi') : [(0, 64, 3)]
('rpc5', 'eta') : [(0, 32, 3)]
('rpc5', 'phi') : [(0, 64, 4)]
0


In [64]:
show(eta_df0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [137]:
eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]
phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]
rpc_number = ['rpc0', 'rpc1', 'rpc2', 'rpc3', 'rpc4', 'rpc5']
for i in range(6):
    eta_df[i]['TDC Number'] = eta_df[i].apply(
        lambda row: compute_tdc_number(rpc_number[i], row['Direction'], -row['expected hits'], reversed_mapping)
        if pd.notna(row['expected hits']) else None,
        axis = 1
    )
    phi_df[i]['TDC Number'] = phi_df[i].apply(
        lambda row: compute_tdc_number(rpc_number[i], row['Direction'], row['expected hits'], reversed_mapping)
        if pd.notna(row['expected hits']) else None,
        axis = 1
    )

    

In [138]:
combined_eta_df = pd.concat(eta_df, ignore_index=True)
combined_phi_df = pd.concat(phi_df, ignore_index=True)

# Now concatenate the two resulting DataFrames vertically as well
combined_df = pd.concat([combined_eta_df, combined_phi_df], ignore_index=True)

In [111]:
show(combined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [112]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7542 entries, 0 to 7541
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Event Number    7542 non-null   int64  
 1   Direction       7542 non-null   object 
 2   expected hits   7542 non-null   float64
 3   rpc_number      5388 non-null   object 
 4   midpoint        5388 non-null   float64
 5   hit_difference  5388 non-null   float64
 6   TDC Number      7542 non-null   int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 412.6+ KB
None


In [139]:
tdc_numbers = combined_df['TDC Number'].unique()  

tdc_dfs = {}

for tdc_number in tdc_numbers:
    tdc_dfs[tdc_number] = combined_df[combined_df['TDC Number'] == tdc_number]

df_tdc_0 = tdc_dfs[0]

df_tdc_1 = tdc_dfs[1]
df_tdc_2 = tdc_dfs[2]
df_tdc_3 = tdc_dfs[3]
df_tdc_4 = tdc_dfs[4]

In [115]:
show(df_tdc_0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [140]:
thresholds = 3
df_tdc = [df_tdc_0, df_tdc_1, df_tdc_2, df_tdc_3, df_tdc_4]

efficiencies = []

label_tdc = ['tdc0', 'tdc1', 'tdc2', 'tdc3', 'tdc4']

for i in range(5):
    valid_hits = df_tdc[i].dropna(subset=['hit_difference'])
    count_valid = valid_hits[valid_hits['hit_difference'].abs() <= thresholds].shape[0]
    total_valid = df_tdc[i].shape[0]
    efficiency = count_valid / total_valid if total_valid > 0 else 0
    efficiencies.append(efficiency)

plt.figure(figsize=(10, 6))
plt.bar(label_tdc, efficiencies, color='blue')
plt.title('Efficiency vs TDC Number for ETA Direction any 5 AtlasON')
plt.xlabel('TDC Number')
plt.ylabel('Efficiency')
plt.ylim(0, 1)  # Set y-axis limits to show from 0 to 100% efficiency
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()